**Import necessary library**

In [8]:
import requests
import certifi
import json
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import FixedLocator
from IPython.display import display, HTML
import numpy as np

**Replace with your API key**

In [9]:
api_key = "255446bcdde7ca9fe776258d09e8411bbb8d1cade2ebd6aba440f80f6817c3fd"

**EX1: The top 10 6-grams in the 20 plays by frequency**  

**Text set = Shakespeare 20 for demos**  
**Segment by text**  
**Ngrams = 6**  
**Output Size = 10**  

In [ ]:
#Character to exlude from search
excludeWords = ["[","\\", "]", "_", "`", "!", "\"", "#", "%", "'", "(", ")", "+", ",", "-", "–", ".", "/", ":", ";", "{", "|", "}", "=", "~", "?" ]

# Replace with your actual text set ID
textset_id = 86 

request_url = "https://sia.ardc-hdcl-sia-iaw.cloud.edu.au/api/v1/word-frequencies"
word_freq_request = {
    'textSet': textset_id,
    'option':{
        'blockMethod': 0,       # Segment by text
        'numberOfNGrams' : 6,
        'outputSize': 10,
        'excludeWords': excludeWords,
    }
}

# Make the API Request
response = requests.post(request_url, json=word_freq_request, headers={"X-API-KEY": api_key}, timeout=1200)

# Create a dictionary to hold word frequencies
word_frequency_map = {}
word_frequency_text_map = {}
word_type_map = {}
word_size_map = {}

# Handle the Response
if response.status_code == 200:
    response_data = response.json()
    blocks = response_data.get("blocks", [])

    for block in blocks:
        freqs = block.get('frequencies', [])
        name = block.get('name' , 'Unknown')

        for freq in freqs:
            word = freq.get('word')
            value = freq.get('value')

            word_frequency_map[word] = word_frequency_map.get(word, 0) + value
            
            # Store the frequency specific to this block
            if word not in word_frequency_text_map:
                word_frequency_text_map[word] = {}
            word_frequency_text_map[word][name] = value
            
        word_type_map[name] = block.get('uniqueWordCount' , 0)
        word_size_map[name] = block.get('size' , 0)
            
            
    # Sort by frequency and take the top 20 words
    sorted_items = sorted(word_frequency_map.items(), key=lambda x: x[1], reverse=True)
    sorted_words = [item[0] for item in sorted_items]
    sorted_frequencies = [item[1] for item in sorted_items]

    # Plotting
    plt.figure(figsize=(12, 6))
    plt.bar(sorted_words, sorted_frequencies, color='blue')
    plt.xlabel('Words')
    plt.ylabel('Frequency')
    plt.title('Top 10 6-grams in the 20 plays by frequency ')
    plt.xticks(rotation=70)
    plt.show()
    
    # Table dispaly
    html = '<div style="overflow-x: auto; margin-top: 40px;"><table border="1">'
    # Table header
    html += '<tr><th>Word</th>'
    for block in blocks:
        column_name = block.get('name', '')  
        html += f'<th style="white-space: nowrap;">{column_name}</th>'
    html += '</tr>'

    # Adding data rows
    for word in sorted_words:
        html += f"<tr><td>{word}</td>"
        for block in blocks:
            column_name = block.get('name', '')
            frequency = word_frequency_text_map.get(word, {}).get(column_name, 0)
            html += f"<td>{frequency}</td>"
        html += "</tr>"
    
    # Adding Word Types row
    html += "<tr><td>Word Types</td>"
    for block in blocks:
        column_name = block.get('name', '')
        word_types = word_type_map.get(column_name,0)
        html += f"<td>{word_types}</td>"
    html += "</tr>"
        
    # Adding Sizes row
    html += "<tr><td>Size</td>"
    for block in blocks:
        column_name = block.get('name', '')
        size = word_size_map.get(column_name,0)
        html += f"<td>{size}</td>"
    html += "</tr>"

    html += '</table></div>'

    display(HTML(html))

else:
    print(f"Failed: {response.status_code} {response.reason}")